__First use the score test to evalute model with qq plot in plot_pcangsd_assoc.ipynb, then use the EM model to get effect sizes__

__make an index for the reference fasta__

In [ ]:
%cd /data/katrinac/APPC_sequencing/Genome/ensembl/
#!mv /data/katrinac/APPC_sequencing/Genome/ensembl/Amphiprion_percula.Nemo_v1.dna.toplevel.fa /data/katrinac/APPC_sequencing/Genome/ensembl/Amphiprion_percula.Nemo_v1.dna.toplevel.fasta #just to rename so samtools recognizes
!samtools faidx /data/katrinac/APPC_sequencing/Genome/ensembl/Amphiprion_percula.Nemo_v1.dna.toplevel.fasta

In [ ]:
%cd /data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/genotypes/

In [ ]:
!angsd -yBin angsd/case_control_pheno.ybin -doAsso 1 -doPost 1 -GL 1 -doMajorMinor 1 -doMaf 1 -nThreads 30 -minQ 30 -minMapQ 30 -SNP_pval 1e-6 -minInd 16 -doHWE 1 -minHWEpval 0.05 -minMAF 0.1  -fai /data/katrinac/APPC_sequencing/Genome/ensembl/Amphiprion_percula.Nemo_v1.dna.toplevel.fasta.fai -out angsd/bin_assoc1_maf0.1 -bam bamlist.list 

__permute phenotypes for significance testing and concatenate the permuted output__

In [ ]:
%cd angsd/permute/

In [ ]:
!pwd

In [ ]:
!seq 100 > permute.txt

In [ ]:
%cd /data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/genotypes/

__make a file with the header to concatenate results to__

In [ ]:
!echo -e "Chromosome\tPosition\tMajor\tMinor\tFrequency\tN\tLRT\thigh_WT/HE/HO" > angsd/permute/assoc2/cat_permuted_assoc2_bin.lrt

__Do the gwas 100 times, resampling the phenotype with replacement__

In [ ]:
#in the permute_angsd env
#conda activate permute_angsd
for i in $(cat angsd/permute/permute.txt); do Rscript ~/ClownfishGWAS/bioinformatics_scripts/perm_phenotype.R; angsd -yBin angsd/permute/case_control_pheno_NULL.ybin -doAsso 2 -doMaf 4 -nThreads 10 -minCount 16  -beagle angsd/ensembl_appc_angsd_probs_minQ30_maf.5.beagle.gprobs.gz -fai /data/katrinac/APPC_sequencing/Genome/ensembl/Amphiprion_percula.Nemo_v1.dna.toplevel.fasta.fai -out angsd/permute/assoc2/bin_assoc2_null$i; gunzip angsd/permute/assoc2/bin_assoc2_null$i.lrt0.gz; sed  -e '1d' angsd/permute/assoc2/bin_assoc2_null$i.lrt0 >> angsd/permute/assoc2/cat_permuted_assoc2_bin.lrt; gzip angsd/permute/assoc2/bin_assoc2_null$i.lrt0; done 

#gunzip angsd/permute/assoc2/bin_assoc1_null$i.lrt0.gz; sed  -e '1d' angsd/permute/assoc2/bin_assoc1_null$i.lrt0 >> angsd/permute/assoc2/cat_permuted_assoc2_bin.lrt; gzip angsd/permute/assoc2/bin_assoc1_null$i.lrt0
#gzip angsd/permute/*lrt0
##angsd -yBin angsd/permute/case_control_pheno_null.ybin -doAsso 1 -doPost 1 -GL 1 -doMajorMinor 1 -doMaf 1 -nThreads 40 -minQ 30 -minMapQ 30 -SNP_pval 1e-6 -minInd 16 -minMAF 0.05  -fai /data/katrinac/APPC_sequencing/Genome/ensembl/Amphiprion_percula.Nemo_v1.dna.toplevel.fasta.fai -out angsd/permute/bin_assoc1_null$i -bam bamlist.list | 


__Get the GWAS assoc2 results for the true association__

__Same for ucrit as a quantitative phenotype__

In [254]:
!echo -e "Chromosome\tPosition\tMajor\tMinor\tFrequency\tN\tLRT\thigh_WT/HE/HO" > angsd/permute/assoc2/cat_permuted_assoc2_ucrit.lrt

In [ ]:

 for i in $(cat angsd/permute/permute.txt); do Rscript ~/ClownfishGWAS/bioinformatics_scripts/perm_phenotype_ucrit.R; angsd -yQuant angsd/permute/ucrit_pheno_NULL.yquant -doAsso 2 -doMaf 4 -nThreads 30 -minCount 16 -beagle angsd/ensembl_appc_angsd_probs_noMaxdepth_maf0.05_minQ30.beagle.gprobs.gz -fai /data/katrinac/APPC_sequencing/Genome/ensembl/Amphiprion_percula.Nemo_v1.dna.toplevel.fasta.fai -out angsd/permute/assoc2/ucrit_assoc2_null$i; gunzip angsd/permute/assoc2/ucrit_assoc2_null$i.lrt0.gz; sed  -e '1d' angsd/permute/assoc2/ucrit_assoc2_null$i.lrt0 >> angsd/permute/assoc2/cat_permuted_assoc2_ucrit.lrt; gzip angsd/permute/assoc2/ucrit_assoc2_null$i.lrt0; done 


In [2]:
#05/18/2022 try Ucrit again with PC1 as a covariate
!echo -e "Chromosome\tPosition\tMajor\tMinor\tFrequency\tN\tLRT\thigh_WT/HE/HO" > angsd/permute/ucrit_PC1/cat_permuted_assoc2_ucrit.lrt
for i in $(cat angsd/permute/permute.txt); do Rscript ~/ClownfishGWAS/bioinformatics_scripts/perm_phenotype_ucrit.R; angsd -yQuant angsd/permute/ucrit_pheno_NULL.yquant -cov angsd/pc1_unlinked.cov -doAsso 2 -doMaf 4 -nThreads 30 -minCount 16 -beagle angsd/ensembl_appc_angsd_probs_noMaxdepth_maf0.05_minQ30.beagle.gprobs.gz -fai /data/katrinac/APPC_sequencing/Genome/ensembl/Amphiprion_percula.Nemo_v1.dna.toplevel.fasta.fai -out angsd/permute/ucrit_PC1/ucrit_assoc2_null$i; gunzip angsd/permute/assoc2/ucrit_assoc2_null$i.lrt0.gz; sed  -e '1d' angsd/permute/ucrit_PC1/ucrit_assoc2_null$i.lrt0 >> angsd/permute/ucrit_PC1/cat_permuted_assoc2_ucrit.lrt; gzip angsd/permute/ucrit_PC1/ucrit_assoc2_null$i.lrt0; done 


SyntaxError: invalid syntax (888356630.py, line 3)

In [252]:
!angsd -yQuant angsd/Ucrit_pheno.yquant -doAsso 2 -doMaf 4 -nThreads 30 -minCount 16 -beagle angsd/ensembl_appc_angsd_probs_noMaxdepth_maf0.05_minQ30.beagle.gprobs.gz -fai /data/katrinac/APPC_sequencing/Genome/ensembl/Amphiprion_percula.Nemo_v1.dna.toplevel.fasta.fai -out angsd/ucrit_assoc2_maf0.05_noMaxdepth   

	-> angsd version: 0.933 (htslib: 1.9) build(May  6 2020 21:25:11)
	-> Printing at chr: PYAM01000342.1 pos:13712 chunknumber 142500 contains 2 sitessesDone reading beagle
	-> Done reading data waiting for calculations to finish
	-> Waiting for nThreads:3
	-> Done waiting for threads
	-> Output filenames:
		->"angsd/ucrit_assoc2_maf0.05_noMaxdepth_minCount16.arg"
		->"angsd/ucrit_assoc2_maf0.05_noMaxdepth_minCount16.mafs.gz"
		->"angsd/ucrit_assoc2_maf0.05_noMaxdepth_minCount16.lrt0.gz"
	-> Thu Apr  7 09:45:52 2022
	-> Arguments and parameters for all analysis are located in .arg file
	-> Total number of sites analyzed: 7116048
	-> Number of sites retained after filtering: 7116048 
	[ALL done] cpu-time used =  562.07 sec
	[ALL done] walltime used =  501.00 sec


In [261]:
!angsd -yQuant angsd/Ucrit_pheno.yquant -doAsso 5 -doMaf 4 -minCount 16 -nThreads 5 -beagle angsd/ensembl_appc_angsd_probs_noMaxdepth_maf0.05_minQ30.beagle.gprobs.gz -fai /data/katrinac/APPC_sequencing/Genome/ensembl/Amphiprion_percula.Nemo_v1.dna.toplevel.fasta.fai -out angsd/ucrit_assoc5_maf0.05_noMaxdepth_minQ30_minCount16   

	-> angsd version: 0.933 (htslib: 1.9) build(May  6 2020 21:25:11)
	-> Printing at chr: PYAM01000348.1 pos:7715 chunknumber 139600 contains 6 sitesDone reading beagle
	-> Done reading data waiting for calculations to finish
	-> Waiting for nThreads:3
	-> Done waiting for threads
	-> Output filenames:
		->"angsd/ucrit_assoc5_maf0.05_noMaxdepth_minQ30_minCount16.arg"
		->"angsd/ucrit_assoc5_maf0.05_noMaxdepth_minQ30_minCount16.mafs.gz"
		->"angsd/ucrit_assoc5_maf0.05_noMaxdepth_minQ30_minCount16.lrt0.gz"
	-> Tue Apr 12 10:01:00 2022
	-> Arguments and parameters for all analysis are located in .arg file
	-> Total number of sites analyzed: 6971649
	-> Number of sites retained after filtering: 6971649 
	[ALL done] cpu-time used =  609.52 sec
	[ALL done] walltime used =  514.00 sec


In [263]:
!angsd -yBin angsd/case_control_pheno.ybin -doAsso 5 -doMaf 4 -minCount 16 -nThreads 5 -beagle angsd/ensembl_appc_angsd_probs_noMaxdepth_maf0.05_minQ30.beagle.gprobs.gz -fai /data/katrinac/APPC_sequencing/Genome/ensembl/Amphiprion_percula.Nemo_v1.dna.toplevel.fasta.fai -out angsd/bin_assoc5_maf0.05_noMaxdepth_minQ30_minCount16   

	-> angsd version: 0.933 (htslib: 1.9) build(May  6 2020 21:25:11)
	-> Printing at chr: PYAM01000348.1 pos:7715 chunknumber 139600 contains 6 sitesDone reading beagle
	-> Done reading data waiting for calculations to finish
	-> Waiting for nThreads:3
	-> Done waiting for threads
	-> Output filenames:
		->"angsd/bin_assoc5_maf0.05_noMaxdepth_minQ30_minCount16.arg"
		->"angsd/bin_assoc5_maf0.05_noMaxdepth_minQ30_minCount16.mafs.gz"
		->"angsd/bin_assoc5_maf0.05_noMaxdepth_minQ30_minCount16.lrt0.gz"
	-> Tue Apr 12 10:25:34 2022
	-> Arguments and parameters for all analysis are located in .arg file
	-> Total number of sites analyzed: 6971649
	-> Number of sites retained after filtering: 6971649 
	[ALL done] cpu-time used =  605.68 sec
	[ALL done] walltime used =  518.00 sec


__Finally with SL as a quantitative phenotype__

In [255]:
!echo -e "Chromosome\tPosition\tMajor\tMinor\tFrequency\tN\tLRT\thigh_WT/HE/HO" > angsd/permute/assoc2/cat_permuted_assoc2_sl.lrt

In [ ]:
for i in $(cat angsd/permute/permute.txt); do Rscript ~/ClownfishGWAS/bioinformatics_scripts/perm_phenotype_sl.R; angsd -yQuant angsd/permute/sl_pheno_NULL.yquant -doAsso 2 -doMaf 4 -minCount 16 -cov angsd/pc1_unlinked.cov -nThreads 30 -beagle angsd/ensembl_appc_angsd_probs_noMaxdepth_maf0.05_minQ30.beagle.gprobs.gz -fai /data/katrinac/APPC_sequencing/Genome/ensembl/Amphiprion_percula.Nemo_v1.dna.toplevel.fasta.fai -out angsd/permute/assoc2/sl_assoc2_null$i; gunzip angsd/permute/assoc2/sl_assoc2_null$i.lrt0.gz; sed  -e '1d' angsd/permute/assoc2/sl_assoc2_null$i.lrt0 >> angsd/permute/assoc2/cat_permuted_assoc2_sl.lrt; gzip angsd/permute/assoc2/sl_assoc2_null$i.lrt0; done 


In [262]:
!angsd -yQuant angsd/SL_pheno.yquant -doAsso 2 -doMaf 4 -nThreads 5 -minCount 16 -cov angsd/pc1_unlinked.cov -beagle angsd/ensembl_appc_angsd_probs_noMaxdepth_maf0.05_minQ30.beagle.gprobs.gz -fai /data/katrinac/APPC_sequencing/Genome/ensembl/Amphiprion_percula.Nemo_v1.dna.toplevel.fasta.fai -out angsd/sl_assoc5_maf0.05_noMaxdepth_unlinkedPC1cov_minQ30_minCount16  

	-> angsd version: 0.933 (htslib: 1.9) build(May  6 2020 21:25:11)
	-> Printing at chr: PYAM01000348.1 pos:7715 chunknumber 139600 contains 6 sitesDone reading beagle
	-> Done reading data waiting for calculations to finish
	-> Waiting for nThreads:1
	-> Done waiting for threads
	-> Output filenames:
		->"angsd/sl_assoc5_maf0.05_noMaxdepth_unlinkedPC1cov_minQ30_minCount16.arg"
		->"angsd/sl_assoc5_maf0.05_noMaxdepth_unlinkedPC1cov_minQ30_minCount16.mafs.gz"
		->"angsd/sl_assoc5_maf0.05_noMaxdepth_unlinkedPC1cov_minQ30_minCount16.lrt0.gz"
	-> Tue Apr 12 10:09:36 2022
	-> Arguments and parameters for all analysis are located in .arg file
	-> Total number of sites analyzed: 6971649
	-> Number of sites retained after filtering: 6971649 
	[ALL done] cpu-time used =  617.11 sec
	[ALL done] walltime used =  516.00 sec


__use the bam files to get the true phenotypes model fit- NO UNHELPFUL__